In [24]:
import numpy as np
from gurobipy import *


In [25]:
M = 2
#supply

N = 3
#demand

T = 2
#period

num_i = M
num_j = N
num_t = T

# I = np.array([1,1])
# I_binary,initial solution for benders


Z = np.array([400
            for _ in range(num_i)])
# Z'i_capacity

zeta = np.array([
[180,210,240],
[240,210,180]
])
# zeta'tj_demand

eta = 50
# eta_price

c =np.array([30,35])
# ci_productioncost

C =np.array([10,10,10])
# Cj_inventory cost

d= np.array([
[5,3,8],
[4,7,5]
])
# dij_transportationcost

F= np.array([20,20])
#Fi_fixed construction cost

f=np.array([[0.7
    for j in range(num_j)]
    for t in range(num_t)])
#ftj_target of fill rate


R=0
#target profit

num_k = 3
a = [0.3909, 0.01298,0]
b = [0, 0.8701, 1]
#piecewise linear function,k=3

params = dict()
params["num_i"] = num_i
params["num_j"] = num_j
params["num_t"] = num_t
params["Z"] = Z
#params["P"] = P
params["zeta"] = zeta
params["eta"] = eta
params["c"] = c
params["C"] = C
params["d"] = d
params["F"] = F
params["f"] = f
params["R"] = R
params["a"] = a
params["b"] = b



In [26]:
mip = Model()
#决策变量
y = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)], 
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="y")
P = mip.addVars([t for t in range(num_t)],
                [i for i in range(num_i)],
                vtype=GRB.CONTINUOUS, name="P")
I = mip.addVars([i for i in range(num_i)], 
                vtype=GRB.BINARY, name="I")
l = mip.addVars([t for t in range(num_t)],
                [j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="l")
D = mip.addVars([j for j in range(num_j)],
                vtype=GRB.CONTINUOUS, name="D")

#目标函数
mip.setObjective(
    (quicksum(D[j]
        for j in range(num_j))*eta
     -quicksum((d[i][j])*y[t,i,j]
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -quicksum(c[i]*P[t,i]
        for t in range(num_t)
        for i in range(num_i))
     -quicksum(F[i]*I[i] 
        for i in range(num_i))
     -quicksum(C[j]*l[t,j]
        for t in range(num_t)
        for j in range(num_j))
    ),GRB.MAXIMIZE
)

mip.update()

In [27]:
#约束
c0 = mip.addConstrs(
    quicksum(
        (quicksum(
            y[n,i,j] for i in range(num_i)
        )-zeta[n,j]
        )
    for n in range(t)
    )
    +quicksum(y[t,i,j] for i in range(num_i))
    -f[t][j]*zeta[t][j]
    >=0
    for j in range(num_j)
    for t in range(num_t)
)
#所有涉及n的地方都要+1
#需求满足率约束

c1=mip.addConstrs(
    (D[j]
     -quicksum(zeta[t,j]
               for t in range(num_t))
    )<=0
    for t in range(num_t)
    for j in range(num_j)
)

c2=mip.addConstrs(
    (D[j]
     -quicksum(y[t,i,j] 
               for i in range(num_i)
               for t in range(num_t))
    )<=0
    for t in range(num_t)
    for j in range(num_j)
)
#销售量取小

c3=mip.addConstrs(
    l[t,j]
    -quicksum(
        (quicksum(
            y[n,i,j] for i in range(num_i)
        )-zeta[n,j]) 
        for n in range(t+1)
    ) >= 0
    for j in range(num_j)
    for t in range(num_t)
)
#所有涉及n的地方都要+1
#库存取正

c4 = mip.addConstrs( 
    quicksum(
        y[t,i,j] for j in range(num_j)
    ) <= P[t,i]
    for i in range(num_i)
    for t in range(num_t)
)

c5 = mip.addConstrs( 
    P[t,i]-Z[i]*I[i]<=0
    for i in range(num_i)
    for t in range(num_t)
)

mip.optimize()

# Print solution
print("Optimal solution of decision variables")
for v in mip.getVars():
    print (v.varName, v.x)

obj = mip.getObjective()
print("Objective value")
print(obj.getValue())

Optimize a model with 32 rows, 27 columns and 102 nonzeros
Variable types: 25 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 16 rows and 6 columns
Presolve time: 0.00s
Presolved: 16 rows, 21 columns, 57 nonzeros
Variable types: 21 continuous, 0 integer (0 binary)

Root relaxation: objective 1.742000e+04, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    17420.000000 17420.0000  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 17420 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.742000000000e+04, best bound 1.742000000000e+04, gap 0.0000%
Optimal solution of de

In [28]:
fullfill=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
f=np.array([[1.00 for j in range(num_j)]for t in range(num_t)])
for t in range(num_t):
    for j in range(num_j):
        #fullfill[t,j]=sum(y[t,i,j] for i in range(num_i))
        fullfill[t,j]=sum(
            (sum(y[n,i,j].X for i in range(num_i))-zeta[n,j])
            for n in range(t)
        )+sum(y[t,i,j].X for i in range(num_i))
        f[t,j]=fullfill[t,j]/zeta[t,j]
print("fullfilled demand")
print(fullfill)
print("actual demand")
print(zeta)
print("fill rate")
print(f)
###################################
L=np.array([[0 for j in range(num_j)]for t in range(num_t)])
for t in range(num_t):
    for j in range(num_j):
        L[t,j]=sum(
        (sum(
            y[n,i,j].x for i in range(num_i)
        )-zeta[n,j]) 
        for n in range(t+1)
    )
print("Inventory level of each store")
print(L)  
###################################
profit=(
     sum(D[j].x
        for j in range(num_j))*eta
     -sum((d[i][j])*y[t,i,j].x
        for t in range(num_t)
        for i in range(num_i)
        for j in range(num_j))
     -sum(c[i]*P[t,i].x
        for t in range(num_t)
        for i in range(num_i))
     -sum(F[i]*I[i].x
        for i in range(num_i))
     -sum(C[j]*l[t,j].x
        for t in range(num_t)
        for j in range(num_j))
    )
print("profit")
print(profit)

fullfilled demand
[[ 180.  210.  168.]
 [ 240.  210.  180.]]
actual demand
[[180 210 240]
 [240 210 180]]
fill rate
[[ 1.   1.   0.7]
 [ 1.   1.   1. ]]
Inventory level of each store
[[  0   0 -72]
 [  0   0   0]]
profit
17420.0
